# Multi-Class Classification Iris Flowers Project 2 for Mothers who love Gardening and Flowers:

# Identifying Flower Types

# By Sequential API

In [75]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Next, we need to initialize the random number generator to a constant value (7).

In [76]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [77]:
# load dataset
dataframe = pd.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [78]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = utils.to_categorical(encoded_Y)

# Baseline Model

In [83]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(layers.Dense(8, activation = 'relu', input_shape = (4,)))
    model.add(layers.Dense(3, activation = 'softmax'))

    # Compile model
    model.compile(optimizer = 'adam',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    
    return model


In [84]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)


In [85]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)


In [86]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 96.00% (4.42%)


# Smaller Model

In [87]:
# define smaller model
def smaller_model():
    # create model
    model = Sequential()
    model.add(layers.Dense(4, activation = 'relu', input_shape = (4,)))
    model.add(layers.Dense(3, activation = 'softmax'))

    # Compile model
    model.compile(optimizer = 'adam',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    
    return model


In [88]:
estimator = KerasClassifier(build_fn=smaller_model, epochs=200, batch_size=5, verbose=0)


In [89]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)


In [90]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 97.33% (3.27%)


# Larger Model

In [91]:
# define large model
def large_model():
    # create model
    model = Sequential()
    model.add(layers.Dense(16, activation = 'relu', input_shape = (4,)))
    model.add(layers.Dense(3, activation = 'softmax'))

    # Compile model
    model.compile(optimizer = 'adam',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    
    return model

In [92]:
estimator = KerasClassifier(build_fn=large_model, epochs=200, batch_size=5, verbose=0)


In [93]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)


In [94]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 96.67% (4.47%)


# Overfit Model

In [96]:
# define overfit model
def overfit_model():
    # create model
    model = Sequential()
    model.add(layers.Dense(16, activation = 'relu', input_shape = (4,)))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(3, activation = 'softmax'))

    # Compile model
    model.compile(optimizer = 'adam',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    
    return model

In [97]:
estimator = KerasClassifier(build_fn=overfit_model, epochs=300, batch_size=20, verbose=0)


In [98]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)


In [99]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Overfit: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Overfit: 96.00% (5.33%)


# Without Using Scikit-Learn

# K-Fold Cross Validation

In [101]:
kf = 15
num_val_samples = len(X) // kf
num_epochs = 100
all_scores = []

In [102]:
for i in range(kf):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [encoded_Y[:i * num_val_samples],
         encoded_Y[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = baseline_model()
    
    model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=5, verbose=0)
    val_err, val_acc = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9
processing fold # 10
processing fold # 11
processing fold # 12
processing fold # 13
processing fold # 14


In [103]:
all_scores


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.7, 0.9, 1.0, 1.0, 1.0, 0.9, 0.8, 1.0]

In [104]:
###### all_scores
print("K-Fold: %.2f%% (%.2f%%)" % (np.mean(all_scores)*100, np.std(all_scores)*100))


K-Fold: 94.67% (8.84%)
